<h2>Assignment_MM02_B09705039_劉惟恩

In [1]:
import numpy as np
import pandas as pd

<h3>(a)Simulating data

<h5>purpose : Simulate the data of 2000 NTU student's independent variable and dependent variable about calculus.<br>
    result : 2000 data established.

In [2]:
np.random.seed(1234)
n = 2000 # number of NTU calc I students

# simulate high school math GPA
hs_math_gpa = np.array(np.random.normal(loc = 3.5, scale = 0.5, size = n))
# truncate to 2.0-4.0
hs_math_gpa = np.array(np.where(hs_math_gpa < 2.0, 2.0,
                      np.where(hs_math_gpa > 4.0, 4.0, hs_math_gpa)))

# simulate previous calculus in high school:
# - 75% of people with HS math GPAs over 3.6 took HS calculus
# - 40% of people with HS math GPAs under 3.6 took HS calculus
# binomial data with 1 trial per student, probabilities as above
hs_calculus = np.array(np.random.binomial(1, size = n,
                      p = np.where(hs_math_gpa >= 3.6, 0.75, 0.40)))

# simulate precalculus at NTU:
# - if no high school calculus, 70% take precalculus, regardless of grade
# - if high school calculus, 60% take precalculus if HS grade < 3.5,
#   and 25% otherwise
NTU_precalc = np.array(np.random.binomial(1, size = n,
                     p = np.where(hs_calculus == 0, 0.70,
                                np.where(hs_math_gpa < 3.5, 0.60, 0.25))))

In [3]:
true_beta = { "Intercept" : 0.3,
              "hs_math_gpa" : 0.7,
              "hs_calculus" : 0.3,
              "NTU_precalc" : 0.1 }

true_sigma = 0.5
NTU_calculus_gpa = np.array(true_beta["Intercept"] + true_beta["hs_math_gpa"] * hs_math_gpa + true_beta["hs_calculus"] * hs_calculus + true_beta["NTU_precalc"] * NTU_precalc 
                            + np.random.normal(loc = 0, scale = true_sigma, size = n))
# we don't see exact GPAs, just nearest 0.1
NTU_calculus_gpa = np.array(np.round(NTU_calculus_gpa, 1))
# truncate under 0.7 or over 4.0
NTU_calculus_gpa = np.array(np.where(NTU_calculus_gpa > 4.0, 4.0,
                          np.where(NTU_calculus_gpa < 0.0, 0.0,
                                 np.where(NTU_calculus_gpa < 0.7, 0.7,
                                        NTU_calculus_gpa))))

<h3>(b)Linear regression

<h5>purpose : Using the data established above to do linear regression.<br>
    result : Print out the OLS Regression Results.

In [4]:
d1 = {'NTU_calculus_gpa': NTU_calculus_gpa, 'hs_math_gpa': hs_math_gpa, 'hs_calculus' : hs_calculus, 'NTU_precalc' : NTU_precalc}

calculus_data = pd.DataFrame (data = d1)
calculus_data

,NTU_calculus_gpa,hs_math_gpa,hs_calculus,NTU_precalc
0,2.9,3.735718,1,1
1,2.8,2.904512,0,1
2,3.2,4.000000,0,1
3,2.4,3.343674,0,0
4,3.0,3.139706,1,1
...,...,...,...,...
1995,3.6,3.816289,1,0
1996,3.1,4.000000,1,0
1997,3.0,3.890731,1,0
1998,3.7,4.000000,1,0


In [5]:
from sklearn.linear_model import LinearRegression
calculus_lm = LinearRegression()

X_var = calculus_data[['hs_math_gpa', 'hs_calculus', 'NTU_precalc']]
Y_var = calculus_data['NTU_calculus_gpa']

calculus_lm.fit(X_var, Y_var)

import statsmodels.api as sm

X_var = sm.add_constant(X_var)
results = sm.OLS(Y_var, X_var).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       NTU_calculus_gpa   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     353.0
Date:                Fri, 07 May 2021   Prob (F-statistic):          6.63e-184
Time:                        18:12:52   Log-Likelihood:                -1353.0
No. Observations:                2000   AIC:                             2714.
Df Residuals:                    1996   BIC:                             2736.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.4872      0.095      5.120      0.000       0.301       0.674
hs_math_gpa     0.6372      0.027     23.538      0.000       0.584       0.690
hs_calculus     0.3470      0.024     14.678      0.000       0.301       0.393
NTU_precalc     0.0945      0.023      4.111      0.000       0.049       0.140
==============================================================================
Omnibus:                        8.233   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.016   Jarque-Bera (JB):                8.148
Skew:                          -0.138   Prob(JB):                       0.0170
Kurtosis:                       2.851   Cond. No.                         34.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<h3>(c)Setup X and y

<h5>purpose : Put the stimulated data into X y arrays.<br>
    result : X y arrays established.

In [6]:
allOne = np.ones((2000, 1))
one = pd.DataFrame(allOne, columns = ['Intercept'])

X = pd.concat((one, calculus_data['hs_math_gpa'], calculus_data['hs_calculus'], calculus_data['NTU_precalc']), axis = 1)
print(X)
y = calculus_data['NTU_calculus_gpa']
print(y)

      Intercept  hs_math_gpa  hs_calculus  NTU_precalc
0           1.0     3.735718            1            1
1           1.0     2.904512            0            1
2           1.0     4.000000            0            1
3           1.0     3.343674            0            0
4           1.0     3.139706            1            1
...         ...          ...          ...          ...
1995        1.0     3.816289            1            0
1996        1.0     4.000000            1            0
1997        1.0     3.890731            1            0
1998        1.0     4.000000            1            0
1999        1.0     3.045825            0            1

[2000 rows x 4 columns]
0       2.9
1       2.8
2       3.2
3       2.4
4       3.0
       ... 
1995    3.6
1996    3.1
1997    3.0
1998    3.7
1999    2.7
Name: NTU_calculus_gpa, Length: 2000, dtype: float64


<h3>(d)Compute matrix quantities (adjoint matrix, determinant, inverse)

<h5>purpose : Compute the adjoint matrix, determinant, inverse of X.<br>
    result : Outputs adjoint matrix, determinant, inverse.

In [7]:
C = np.dot(np.transpose(X), X)

A = np.matrix(C)
A = A.getH()
print("adjoint matrix:\n", A)

adjoint matrix:
 [[ 2000.          6941.76090265  1121.          1024.        ]
 [ 6941.76090265 24445.314274    4020.929233    3452.32404242]
 [ 1121.          4020.929233    1121.           402.        ]
 [ 1024.          3452.32404242   402.          1024.        ]]


In [8]:
print("determinant:\n", np.linalg.det(C))

determinant:
 134181610758.34204


In [9]:
I = np.linalg.inv(C)
print("inverse:\n", I)

inverse:
 [[ 0.03990558 -0.01102276  0.00071678 -0.00302474]
 [-0.01102276  0.00322891 -0.00070776  0.00041462]
 [ 0.00071678 -0.00070776  0.00246189  0.00070288]
 [-0.00302474  0.00041462  0.00070288  0.00232753]]


<h3>(e)Compute matrix quantities (beta, residuals, residual variance, standard errors of the covariance matrix)

<h5>purpose : Compute the beta, residuals, residual variance, standard errors of the covariance matrix.<br>
    result : Outputs beta, residuals, residual variance, standard errors of the covariance matrix.

In [10]:
beta = np.linalg.multi_dot([I, np.transpose(X), y])
print("beta:\n", beta)

beta_df = pd.DataFrame(beta).T
beta_df.columns = ['Intercept', 'hs_math_gpa', 'hs_calculus', 'NTU_precalc']

beta:
 [0.48724356 0.6372306  0.34697404 0.09448487]


In [11]:
residuals = y - np.dot(X, beta)
print("residuals:\n", residuals)

residuals:
 0      -0.409216
1       0.367428
2       0.069349
3      -0.217935
4       0.070581
          ...   
1995    0.333926
1996   -0.283140
1997   -0.313511
1998    0.316860
1999    0.177379
Name: NTU_calculus_gpa, Length: 2000, dtype: float64


In [12]:
p = calculus_data.shape[1] - 1

residual_var = np.dot(np.transpose(residuals), residuals / (n - p - 1))
residual_var = pd.to_numeric(residual_var)
print("residual variance:\n", residual_var)

residual variance:
 0.22697720801922477


In [13]:
beta_covar = residual_var * I
beta_SE = np.sqrt(np.diag(beta_covar))
print("standard errors of the covariance matrix:\n", beta_SE)

beta_SE_df = pd.DataFrame(beta_SE).T
beta_SE_df.columns = ['Intercept', 'hs_math_gpa', 'hs_calculus', 'NTU_precalc']

standard errors of the covariance matrix:
 [0.09517172 0.02707193 0.02363878 0.02298468]


<h3>(f)Compute the results (beta comparision)

<h5>purpose : Compare the coefficients of truth, manual and lm.<br>
    result : Outputs comparision dataframe.

In [14]:
true_beta_df = pd.DataFrame (true_beta, index=[0])

coef = results.params
coef_df = pd.DataFrame(data = coef)
coef_df.columns = ['0']
coef_df.index = ['Intercept', 'hs_math_gpa', 'hs_calculus', 'NTU_precalc']

beta_compare = pd.concat((true_beta_df.T, beta_df.T, coef_df), axis = 1)
beta_compare.columns =['Truth', 'Manual', 'ln']
beta_compare

,Truth,Manual,ln
Intercept,0.3,0.487244,0.487244
hs_math_gpa,0.7,0.637231,0.637231
hs_calculus,0.3,0.346974,0.346974
NTU_precalc,0.1,0.094485,0.094485


<h3>(g)Compute the results (standard error comparision)

<h5>purpose : Compare the standard errors of truth, manual and lm.<br>
    result : Outputs comparision dataframe.

In [15]:
true_covar = np.dot(true_sigma**float(2), I)
true_SE = np.sqrt(np.diag(true_covar))
true_SE_df = pd.DataFrame (true_SE, columns = [0], index = ['Intercept', 'hs_math_gpa', 'hs_calculus', 'NTU_precalc'])

SE_df = results.bse
SE_df = SE_df.to_frame()
SE_df.columns = ['0']
SE_df.index = ['Intercept', 'hs_math_gpa', 'hs_calculus', 'NTU_precalc']

SE_compare = pd.concat((true_SE_df, beta_SE_df.T, SE_df), axis = 1)
SE_compare.columns =['Truth', 'Manual', 'ln']
SE_compare

,Truth,Manual,ln
Intercept,0.099882,0.095172,0.095172
hs_math_gpa,0.028412,0.027072,0.027072
hs_calculus,0.024809,0.023639,0.023639
NTU_precalc,0.024122,0.022985,0.022985


<h3>(h)Compute the results (residual variance comparision)

<h5>purpose : Compare the residual variance of truth, manual and lm.<br>
    result : Outputs comparision dataframe.

In [16]:
resid_val = results.resid
resid_val_sum = 0
for i in range(2000):
    resid_val_sum += (resid_val[i] ** 2)
resid_var_lm = resid_val_sum / (n - p - 1)
resid_var_lm_dic = {resid_var_lm}
resid_var_lm_df = pd.DataFrame(resid_var_lm_dic)

resid_var = {residual_var}
resid_var_df =  pd.DataFrame(data = resid_var)

true_resid = {true_sigma**2}
true_resid_df = pd.DataFrame(data = true_resid)

resid_var_compare = pd.concat((true_resid_df, resid_var_df, resid_var_lm_df), axis = 1)
resid_var_compare.columns = ['Truth', 'Manual', 'ln']
resid_var_compare.index = ['residual variance']
resid_var_compare

,Truth,Manual,ln
residual variance,0.25,0.226977,0.226977


<h3>(i)Prediction

<h5>purpose : Predict the calculus gpa of a student and compare truth, manual and lm.<br>
    result : Outputs comparision dataframe.

In [17]:
predi_data = np.array([3.123456, 1, 0])
predi_lm = calculus_lm.predict(predi_data.reshape(1,-1))

predi_truth = true_beta["Intercept"] + true_beta["hs_math_gpa"] * predi_data[0] + true_beta["hs_calculus"] * predi_data[1] + true_beta["NTU_precalc"] * predi_data[2]

predi_man = beta[0] + beta[1] * predi_data[0] + beta[2] * predi_data[1] + beta[3] * predi_data[2]

predi_dic = {'Truth' : predi_truth, 'manual' : predi_man, 'lm' : predi_lm[0]}
predi_compare = pd.DataFrame(data = predi_dic, index = ["NTU calculus GPA"])
predi_compare

,Truth,manual,lm
NTU calculus GPA,2.786419,2.824579,2.824579


<h4> reference : <br>
    http://rebeccaferrell.github.io/CSSS508/Homework/template-HW4-key.html<br>
    https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html<br>
    https://scikit-learn.org/stable/